In [2]:
import torch

In [89]:
N = 2
C = 3
H = 5
W = 5
x = torch.arange(N*H*W).reshape(N, H, W)*2
x

tensor([[[ 0,  2,  4,  6,  8],
         [10, 12, 14, 16, 18],
         [20, 22, 24, 26, 28],
         [30, 32, 34, 36, 38],
         [40, 42, 44, 46, 48]],

        [[50, 52, 54, 56, 58],
         [60, 62, 64, 66, 68],
         [70, 72, 74, 76, 78],
         [80, 82, 84, 86, 88],
         [90, 92, 94, 96, 98]]])

In [96]:
indices = torch.randint(H * W, (N, 8), dtype=torch.long)
indices, indices.shape

(tensor([[19,  3, 23,  8, 22,  8, 12, 22],
         [ 4,  4, 11, 23, 17,  0, 10, 22]]), torch.Size([2, 8]))

In [115]:
ga = torch.gather(x.view(N, H * W), 1, indices).view(N, -1)
ga, ga.shape

(tensor([[38,  6, 46, 16, 44, 16, 24, 44],
         [58, 58, 72, 96, 84, 50, 70, 94]]), torch.Size([2, 8]))

In [98]:
ga = ga / 100
ga, x 

(tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]), tensor([[[ 0,  2,  4,  6,  8],
          [10, 12, 14, 16, 18],
          [20, 22, 24, 26, 28],
          [30, 32, 34, 36, 38],
          [40, 42, 44, 46, 48]],
 
         [[50, 52, 54, 56, 58],
          [60, 62, 64, 66, 68],
          [70, 72, 74, 76, 78],
          [80, 82, 84, 86, 88],
          [90, 92, 94, 96, 98]]]))

In [100]:
x2 = torch.arange(N*C*H*W).reshape(N, C, H, W)*2
x2[0][0], x2[0][1]

(tensor([[ 0,  2,  4,  6,  8],
         [10, 12, 14, 16, 18],
         [20, 22, 24, 26, 28],
         [30, 32, 34, 36, 38],
         [40, 42, 44, 46, 48]]), tensor([[50, 52, 54, 56, 58],
         [60, 62, 64, 66, 68],
         [70, 72, 74, 76, 78],
         [80, 82, 84, 86, 88],
         [90, 92, 94, 96, 98]]))

In [73]:
indices.unsqueeze(1).expand(-1, C, -1).shape

torch.Size([2, 3, 8])

In [78]:
ga = torch.gather(x2.view(N, C, -1), 2, indices.unsqueeze(1).expand(-1, C, -1))
ga, ga.shape

(tensor([[[ 42,  24,  42,  10,  40,  32,  42,  22],
          [ 92,  74,  92,  60,  90,  82,  92,  72],
          [142, 124, 142, 110, 140, 132, 142, 122]],
 
         [[170, 156, 196, 180, 180, 184, 168, 198],
          [220, 206, 246, 230, 230, 234, 218, 248],
          [270, 256, 296, 280, 280, 284, 268, 298]]]), torch.Size([2, 3, 8]))

In [114]:
ga[:, 0, :], ga[:, 1, :]

(tensor([[ 38.,   6.,  46.,  16.,  44.,  16.,  24.,  44.],
         [158., 158., 172., 196., 184., 150., 170., 194.]],
        grad_fn=<SliceBackward>),
 tensor([[ 88.,  56.,  96.,  66.,  94.,  66.,  74.,  94.],
         [208., 208., 222., 246., 234., 200., 220., 244.]],
        grad_fn=<SliceBackward>))

# Gather Backward

In [112]:
x3 = x2.float().requires_grad_(True)
ga = torch.gather(x3.view(N, C, -1), 2, indices.unsqueeze(1).expand(-1, C, -1))
x3.sum().backward()

In [113]:
ga

tensor([[[ 38.,   6.,  46.,  16.,  44.,  16.,  24.,  44.],
         [ 88.,  56.,  96.,  66.,  94.,  66.,  74.,  94.],
         [138., 106., 146., 116., 144., 116., 124., 144.]],

        [[158., 158., 172., 196., 184., 150., 170., 194.],
         [208., 208., 222., 246., 234., 200., 220., 244.],
         [258., 258., 272., 296., 284., 250., 270., 294.]]],
       grad_fn=<GatherBackward>)

In [145]:
N = 2
C = 3
H = 5
W = 5
x = torch.arange(N*H*W).reshape(N, H, W)*2
indices = torch.randint(H * W, (N, 30), dtype=torch.long)
ga = torch.gather(x.view(N, H * W), 1, indices).view(N, -1)
indices, ga

(tensor([[ 5, 18,  8,  5, 10, 15,  3, 22, 24, 11, 24, 19,  8, 13, 16, 18, 18, 11,
          15,  5,  9,  9,  4,  7, 21,  3,  8,  2, 17, 19],
         [ 3, 16,  0, 10,  3, 20,  5,  0,  4, 10, 19,  8, 10, 23, 12, 16, 21,  1,
          23, 15, 23, 21,  5, 24, 23, 11, 10, 15, 19, 19]]),
 tensor([[10, 36, 16, 10, 20, 30,  6, 44, 48, 22, 48, 38, 16, 26, 32, 36, 36, 22,
          30, 10, 18, 18,  8, 14, 42,  6, 16,  4, 34, 38],
         [56, 82, 50, 70, 56, 90, 60, 50, 58, 70, 88, 66, 70, 96, 74, 82, 92, 52,
          96, 80, 96, 92, 60, 98, 96, 72, 70, 80, 88, 88]]))

In [146]:
v, i = ga.topk(5)
print("Value ;", v, v.shape)
print("Idx ;",i, i.shape)

Value ; tensor([[48, 48, 44, 42, 38],
        [98, 96, 96, 96, 96]]) torch.Size([2, 5])
Idx ; tensor([[10,  8,  7, 24, 11],
        [23, 24, 20, 18, 13]]) torch.Size([2, 5])


In [147]:
reverse_indices = torch.gather(indices, 1, i)
reverse_indices

tensor([[24, 24, 22, 21, 19],
        [24, 23, 23, 23, 23]])

In [144]:
torch.gather(x.view(N, H * W), 1, reverse_indices).view(N, -1)

tensor([[36, 36, 34, 26, 12],
        [98, 94, 88, 88, 82]])